In [24]:
import pandas as pd

In [55]:
def calculate_team_stats(path):
    df = pd.read_excel(path)

    required_columns = ['Club', '2FG', '3FG', 'FT', 'Fouls(Rv)', 'PIR']
    if not all(column in df.columns for column in required_columns):
        raise ValueError(f"The file must contain required columns")
    
    df[['2pt_made','2pt_attempted']] = df['2FG'].str.split('/', expand = True).astype(int)
    df[['3pt_made','3pt_attempted']] = df['3FG'].str.split('/', expand = True).astype(int)
    df[['ft_made','ft_attempted']] = df['FT'].str.split('/', expand = True).astype(int)

    df['Fouls(Rv)'] = df['Fouls(Rv)'].astype(int)
    df['PIR'] = df['PIR'].astype(int)

    team_stats = df.groupby('Club').agg({
        '2pt_made': 'sum',
        '3pt_made': 'sum',
        'ft_made': 'sum',
        'Fouls(Rv)':'sum',
        'PIR': 'sum'
    }).reset_index()

    return team_stats


In [56]:
path = 'data/Euroleague_20_21.xlsx'
team_stats = calculate_team_stats(path)

In [57]:
team_stats

,Club,2pt_made,3pt_made,ft_made,Fouls(Rv),PIR
0,AX Armani Exchange Milan,797,387,589,819,3514
1,Alba Berlin,647,342,354,664,2863
2,Anadolu Efes,795,418,609,836,3908
3,Baskonia Vitoria-Gasteiz,685,314,439,678,3183
4,CSKA Moscow,759,388,599,847,3516
5,Crvena Zvezda Mts Belgrade,587,292,470,680,2469
6,FC Barcelona,855,324,598,866,3566
7,FC Bayern Munich,831,281,561,801,3201
8,Fenerbahce Beko Istanbul,781,301,415,685,3106
9,Khimki Moscow Region,620,315,449,630,2626


In [66]:
team_stats['score'] = 47.19 + 0.15 * team_stats['Fouls(Rv)'] + 0.78 * team_stats['2pt_made'] + 1.39 * team_stats['3pt_made'] + 0.54 * team_stats['ft_made']

In [67]:
team_stats

,Club,2pt_made,3pt_made,ft_made,Fouls(Rv),PIR,score
2,Anadolu Efes,795,418,609,836,3908,1702.57
6,FC Barcelona,855,324,598,866,3566,1617.27
0,AX Armani Exchange Milan,797,387,589,819,3514,1647.69
4,CSKA Moscow,759,388,599,847,3516,1629.04
7,FC Bayern Munich,831,281,561,801,3201,1509.05
17,Zenit St Petersburg,705,358,537,803,3266,1505.14
14,Real Madrid,720,400,458,734,3429,1522.21
8,Fenerbahce Beko Istanbul,781,301,415,685,3106,1401.61
15,Valencia Basket,668,318,493,697,3075,1381.02
3,Baskonia Vitoria-Gasteiz,685,314,439,678,3183,1356.71


In [68]:
team_stats = team_stats.sort_values(by=['score']).iloc[::-1]

In [69]:
team_stats 

,Club,2pt_made,3pt_made,ft_made,Fouls(Rv),PIR,score
2,Anadolu Efes,795,418,609,836,3908,1702.57
0,AX Armani Exchange Milan,797,387,589,819,3514,1647.69
4,CSKA Moscow,759,388,599,847,3516,1629.04
6,FC Barcelona,855,324,598,866,3566,1617.27
14,Real Madrid,720,400,458,734,3429,1522.21
7,FC Bayern Munich,831,281,561,801,3201,1509.05
17,Zenit St Petersburg,705,358,537,803,3266,1505.14
8,Fenerbahce Beko Istanbul,781,301,415,685,3106,1401.61
15,Valencia Basket,668,318,493,697,3075,1381.02
3,Baskonia Vitoria-Gasteiz,685,314,439,678,3183,1356.71


In [62]:
team_stats_pir = team_stats.sort_values(by=['PIR']).iloc[::-1]

In [63]:
team_stats_pir

,Club,2pt_made,3pt_made,ft_made,Fouls(Rv),PIR,score
2,Anadolu Efes,795,418,609,836,3908,1593.15
6,FC Barcelona,855,324,598,866,3566,1561.11
4,CSKA Moscow,759,388,599,847,3516,1530.00
0,AX Armani Exchange Milan,797,387,589,819,3514,1553.70
14,Real Madrid,720,400,458,734,3429,1412.61
17,Zenit St Petersburg,705,358,537,803,3266,1414.32
7,FC Bayern Munich,831,281,561,801,3201,1471.08
3,Baskonia Vitoria-Gasteiz,685,314,439,678,3183,1285.05
8,Fenerbahce Beko Istanbul,781,301,415,685,3106,1347.84
15,Valencia Basket,668,318,493,697,3075,1305.36
